In [2]:
import numpy as np
import networkx as nx
import kmapper as km
import sklearn
import warnings
import matplotlib.pyplot as plt
import signal
import time
import random
from networkx.algorithms.similarity import graph_edit_distance

warnings.filterwarnings("ignore")

In [3]:
def select_k(spectrum, minimum_energy = 0.9):
    running_total = 0.0 
    
    total = sum(spectrum)
    if total == 0.0:
        return len(spectrum)
    for i in range(len(spectrum)):
        running_total += spectrum[i]
        if running_total / total >= minimum_energy:
            return i + 1
    return len(spectrum)

In [4]:
def calculate_similarity(graph1,graph2):
    laplacian1 = nx.spectrum.laplacian_spectrum(graph1)
    laplacian2 = nx.spectrum.laplacian_spectrum(graph2)
    
    k1 = select_k(laplacian1)
    k2 = select_k(laplacian2)
    k = min(k1, k2) #k are different between the two graphs, then use the smaller one.
    similarity = sum((laplacian1[:k] - laplacian2[:k])**2) #sum of the squared differences between the largest k eigenvalues
    return similarity


In [5]:
def extract_graph_features(graph):
#     pr = nx.pagerank(graph,0.9)
    dc = nx.degree_centrality(graph)
    cc = nx.closeness_centrality(graph)
    bx = nx.betweenness_centrality(graph)
    c = nx.clustering(graph)
    
    #create list for each features
#     pr_list =  [i for i in pr.values()]
    dc_list =  [i for i in dc.values()]
    cc_list =  [i for i in cc.values()]
    bx_list =  [i for i in bx.values()]
    c_list =  [i for i in c.values()]
    d_list = [val for (node, val) in graph.degree()]
    data = np.column_stack((dc_list,cc_list,bx_list,c_list,d_list))
    return data

In [6]:
def TDA_transformation(data):
    Xfilt = data
    mapper = km.KeplerMapper()
    scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(0, 1))
    Xfilt = scaler.fit_transform(Xfilt)
    lens = mapper.fit_transform(Xfilt, projection=sklearn.manifold.TSNE())
    cls = 2  # We use cls= 5

    graph = mapper.map(lens,Xfilt,clusterer=sklearn.cluster.KMeans(n_clusters=cls,random_state=1618033),
        cover=km.Cover(n_cubes=5, perc_overlap=0.1))
    return km.to_nx(graph)

In [7]:
def add_node_to_graph(graph,p):
    new_node = graph.number_of_nodes() + 1
    graph.add_node(new_node)
    existing_nodes = list(graph.nodes())[:-1]  # Exclude the new node
    for existing_node in existing_nodes:
        if random.random() < p:  
            graph.add_edge(new_node, existing_node)
    return graph
    

In [8]:
def remove_least_degree_node(graph):
    degrees = graph.degree()
    min_degree_node = min(degrees, key=lambda x: x[1])[0]
    # Remove the node with the minimum degree
    graph.remove_node(min_degree_node)
    return graph

In [9]:
def random_add_new_edge(graph):
    node_num = graph.number_of_nodes()
    node1 = random.randint(1,node_num)
    node2 = random.randint(1,node_num)
    while graph.has_edge(node1,node2) or node1 == node2 :
        node1 = random.randint(1,node_num)
        node2 = random.randint(1,node_num)
    graph.add_edge(node1,node2)
    return graph

In [10]:
def random_remove_edge(graph):
    random_edge = random.choice(list(graph.edges()))
    # Remove the randomly selected edge
    graph.remove_edge(*random_edge)
    return graph

In [11]:
def graph_generator(original_graph, node, edge,p):
    new_graph = original_graph.copy()
    if node < 0:
        for i in range(abs(node)):
            new_graph = remove_least_degree_node(new_graph)
    elif node > 0:
        for i in range(abs(node)):
            new_graph = add_node_to_graph(new_graph,p)
    
    if edge < 0:
        for i in range(abs(edge)):
            new_graph = random_remove_edge(new_graph)
    elif edge >0:
        for i in range(abs(edge)):
            new_graph = random_add_new_edge(new_graph)
    return new_graph

In [12]:
def calc_original_average_similarity_for_hop(graph, hop,p):
    counter = 0
    sum = 0
    for i in range(-hop,hop + 1):
        for j in range(-hop,hop + 1):
            if (i == -hop or i == hop or j == -hop or j == hop):
                neighbour = graph_generator(graph,j,i,p)
                score = calculate_similarity(graph, neighbour)
                sum += score
                counter += 1
    return sum/counter

In [13]:
def calc_TDA_average_similarity_for_hop(graph, hop,p):
    counter = 0
    sum = 0
    for i in range(-hop,hop + 1):
        for j in range(-hop,hop + 1):
            if (i == -hop or i == hop or j == -hop or j == hop):
                neighbour = graph_generator(graph,j,i,p)
                TDA_graph = TDA_transformation(extract_graph_features(graph))
                TDA_neighbour = TDA_transformation(extract_graph_features(neighbour))
                score = calculate_similarity(TDA_graph, TDA_neighbour)
                sum += score
                counter += 1
    return sum/counter

In [14]:
def calc_both_average_similarity_for_hop(graph, hop,p):
    counter = 0
    sum_original = 0
    sum_TDA = 0
    for i in range(-hop,hop + 1):
        for j in range(-hop,hop + 1):
            if (i == -hop or i == hop or j == -hop or j == hop):
                neighbour = graph_generator(graph,j,i,p)
                TDA_graph = TDA_transformation(extract_graph_features(graph))
                TDA_neighbour = TDA_transformation(extract_graph_features(neighbour))
                
                score_TDA = calculate_similarity(TDA_graph, TDA_neighbour)
                score_original = calculate_similarity(graph, neighbour)
                
                sum_TDA += score_TDA
                sum_original += score_original
                counter += 1
    return {"TDA": sum_TDA/counter,"original": sum_original/counter}

In [16]:
G = nx.erdos_renyi_graph(n=20,p=0.5)

In [16]:
result_original = calc_original_average_similarity_for_hop(G,2,0.9)
result_original

19.705066461042694

In [17]:
result_TDA = calc_TDA_average_similarity_for_hop(G,1,0.9)
result_TDA

3.625

In [124]:
result_TDA> result_original

False

In [194]:
G = nx.erdos_renyi_graph(n= 30,p=0.1) #n = 5, overlapping 0.1
dict = calc_both_average_similarity_for_hop(G,10,0.1)
TDA = dict['TDA']
original = dict['original']
print (original,TDA)

26.424260497985095 10.274605293822884


In [171]:
score = calc_TDA_average_similarity_for_hop(G,7,0.9)
score

62.28571428571428

In [15]:
def calc_hop():
    threshold = 100
    n = random.randint(30, 50)
    p = round(random.uniform(0, 100))
    reference_graph = nx.erdos_renyi_graph(n,p)
    
    TDA_average = 0
    original_average = 0
    
    TDA_hop = 0
    original_hop = 0
    while TDA_average <= threshold or original_average <= threshold:
        if(TDA_average <= threshold and original_average <= threshold):
            TDA_hop += 1
            original_hop += 1
            result = calc_both_average_similarity_for_hop(reference_graph,TDA_hop ,p)
            TDA_average = result['TDA']
            original_average = result['original']
            
        elif(TDA_average <= threshold):
            TDA_hop += 1
            TDA_average = calc_TDA_average_similarity_for_hop(reference_graph,TDA_hop,p)
        elif(original_average <= threshold):
            original_hop += 1
            original_average = calc_original_average_similarity_for_hop(reference_graph,original_hop,p)

    
    return {"Threshold": threshold, "n":n,"p":p,"last_TDA_average": TDA_average, "last_original_average": original_average,
           "TDA_hop": TDA_hop,"orginal_hop":original_hop}

In [ ]:
result = calc_hop()
result

In [37]:
def calc_hop_v1():
    threshold = 100
    n = random.randint(80,100)
    
    p = round(random.uniform(30, 100))/100
#     n = 30
#     p = 0.6
    reference_graph = nx.erdos_renyi_graph(n,p)
    
    TDA_average = 0
    original_average = 0
    
    TDA_hop = 0
    original_hop = 0
    while (TDA_average <= threshold or original_average <= threshold) and TDA_hop < 10:
        if(TDA_average <= threshold and original_average <= threshold):
            TDA_hop += 1
            original_hop += 1
            result = calc_both_average_similarity_for_hop(reference_graph,TDA_hop ,p)
            TDA_average = result['TDA']
            original_average = result['original']
            print(TDA_average,original_average)
        elif(TDA_average <= threshold):
            TDA_hop += 1
            TDA_average = calc_TDA_average_similarity_for_hop(reference_graph,TDA_hop,p)
            print(TDA_average)
        elif(original_average <= threshold):
            original_hop += 1
            original_average = calc_original_average_similarity_for_hop(reference_graph,original_hop,p)
            

    
    return {"Threshold": threshold, "n":n,"p":p,"last_TDA_average": TDA_average, "last_original_average": original_average,
           "TDA_hop": TDA_hop,"orginal_hop":original_hop}
#     return {"TDA_hop":TDA_hop,"orginnal":original_hop}

In [19]:
result = calc_hop_v1()
result

25.193860090508146 103.17941832798756
25.814374849129177
23.59225980739336
20.56378693141322
25.23307418820466
29.64052264941301
22.662581091961815
29.770285349276335
27.833157828644755
28.88002958981512


{'Threshold': 50,
 'n': 48,
 'p': 0.63,
 'last_TDA_average': 28.88002958981512,
 'last_original_average': 103.17941832798756,
 'TDA_hop': 10,
 'orginal_hop': 1}

In [22]:
result = calc_hop_v1()
resul

33.95747837834349 1.984352623472304
16.69543439802717 8.257591325521911
23.325828771425382 32.003944793009865
33.468608513912784 59.0415088570316
21.67898955915578
25.429875965869684
30.93855982068564
25.073344062365262
24.524953563308376
21.213343072849156


NameError: name 'resul' is not defined

In [24]:
result

{'Threshold': 50,
 'n': 46,
 'p': 0.44,
 'last_TDA_average': 21.213343072849156,
 'last_original_average': 59.0415088570316,
 'TDA_hop': 10,
 'orginal_hop': 4}

In [26]:
result = calc_hop_v1()
resul

12.0 4.3569830386789254
20.25 13.299426233742658
25.151872092638172 22.42561983886885
20.989049392073984 29.429558895042405
20.632973309264578 46.930708689351846
20.45866147132654
20.00663025416645
25.244022286596586
23.01983158993419
21.524833201252175


NameError: name 'resul' is not defined

In [27]:
result

{'Threshold': 40,
 'n': 37,
 'p': 0.45,
 'last_TDA_average': 21.524833201252175,
 'last_original_average': 46.930708689351846,
 'TDA_hop': 10,
 'orginal_hop': 5}

In [29]:
result = calc_hop_v1()
result

12.566804130011953 120.90573354791978
14.73039321881345
13.109146678108738
14.120582269569542
15.467157287525378
18.134407515442813
25.245536677337103
18.95806016476702
20.903388914921383
20.609314575050757


{'Threshold': 35,
 'n': 38,
 'p': 0.79,
 'last_TDA_average': 20.609314575050757,
 'last_original_average': 120.90573354791978,
 'TDA_hop': 10,
 'orginal_hop': 1}

In [31]:
result = calc_hop_v1()
result

26.207478378343488 3.927960801589278
13.046524744255253 12.114885360773897
16.85247637020027 15.267114822646825
16.156333923293353 30.078317567891144
19.36723677732902
18.912836425745763
20.228785292163714
23.63583426054503
25.85344830252786
21.25726682628109


{'Threshold': 30,
 'n': 38,
 'p': 0.43,
 'last_TDA_average': 21.25726682628109,
 'last_original_average': 30.078317567891144,
 'TDA_hop': 10,
 'orginal_hop': 4}

In [38]:
result = calc_hop_v1()
result

40.916739700623296 513.47991885051
24.12393213510389
25.94781197854039
33.8795452549297
33.31390813090973
32.67761848897168
32.619262871888026
39.55643071866573
38.99047026603503
27.59012437032049


{'Threshold': 100,
 'n': 83,
 'p': 0.87,
 'last_TDA_average': 27.59012437032049,
 'last_original_average': 513.47991885051,
 'TDA_hop': 10,
 'orginal_hop': 1}